<a href="https://colab.research.google.com/github/priyal6/gen/blob/main/Prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install torch transformers faiss-cpu sentence-transformers fastapi uvicorn langchain

In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle

documents = [
    "What is RAG?",
    "RAG combines retrieval and generation.",
    "Open-source LLMs are powerful."
]

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents)

In [9]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [10]:
with open('faiss_index.pkl','wb') as f:
  pickle.dump((index, documents), f)

In [11]:
retriever = SentenceTransformer('all-MiniLM-L6-v2')


In [12]:
#load LLM
!pip install transformers --upgrade  # Upgrade to ensure latest version
from transformers import AutoTokenizer, AutoModelForCausalLM #use AutoModelForCausalLM instead

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = AutoModelForCausalLM.from_pretrained(model_name) #use AutoModelForCausalLM instead

In [13]:
def rag_pipeline(query):
  retriever = SentenceTransformer('all-MiniLM-L6-v2')
  query_embedding = retriever.encode([query])
  D, I = index.search(query_embedding, k=2)
  retrieved_docs = [documents[i]for i in I[0]]

  #generating response
  input_text = query + " " + " ".join(retrieved_docs)
  inputs = tokenizer(input_text, return_tensors="pt")
  outputs = generator.generate(**inputs, max_length=100)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

#testing the pipeline
query = "How does RAG work?"
print(rag_pipeline(query))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


How does RAG work? What is RAG? RAG combines retrieval and generation. It is a simple, yet powerful, way to retrieve information from a database. It is also a powerful way to generate a database. RAG is a database that can be used to generate a database.

RAG is a database that can be used to generate a database. RAG is a database that can be used to generate a database. RAG is a database that can be used to generate
